## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Beloha,MG,-25.1667,45.0500,59.00,77,1,10.74,clear sky
1,1,Severo-Kurilsk,RU,50.6789,156.1250,59.34,88,74,3.49,broken clouds
2,2,Esperance,AU,-33.8667,121.9000,58.84,72,3,22.48,clear sky
3,3,Bijie,CN,27.3125,105.2942,72.84,75,7,1.25,clear sky
4,4,Castro,BR,-24.7911,-50.0119,54.99,61,5,3.51,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your minimum temperature preference?")
min_temp = float(input("min_temp: "))

print("What is your maximum temperature preference?")
max_temp = float(input("max_temp: "))

What is your minimum temperature preference?
min_temp: 30
What is your maximum temperature preference?
max_temp: 60


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_cities_df = city_data_df.loc[(city_data_df["Max Temp"] > min_temp) & (city_data_df["Max Temp"] < max_temp)]
   
new_cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Beloha,MG,-25.1667,45.0500,59.00,77,1,10.74,clear sky
1,1,Severo-Kurilsk,RU,50.6789,156.1250,59.34,88,74,3.49,broken clouds
2,2,Esperance,AU,-33.8667,121.9000,58.84,72,3,22.48,clear sky
4,4,Castro,BR,-24.7911,-50.0119,54.99,61,5,3.51,clear sky
5,5,Sorland,NO,67.6670,12.6934,54.14,83,100,4.59,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
new_cities_df.count()

City ID                154
City                   154
Country                150
Lat                    154
Lng                    154
Max Temp               154
Humidity               154
Cloudiness             154
Wind Speed             154
Current Description    154
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_cities_df.dropna()
clean_df

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Beloha,MG,-25.1667,45.0500,59.00,77,1,10.74,clear sky
1,1,Severo-Kurilsk,RU,50.6789,156.1250,59.34,88,74,3.49,broken clouds
2,2,Esperance,AU,-33.8667,121.9000,58.84,72,3,22.48,clear sky
4,4,Castro,BR,-24.7911,-50.0119,54.99,61,5,3.51,clear sky
5,5,Sorland,NO,67.6670,12.6934,54.14,83,100,4.59,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
531,531,Waitara,NZ,-38.9250,174.2500,57.72,89,65,14.45,broken clouds
538,538,Largs,GB,55.7963,-4.8634,59.45,80,100,1.01,overcast clouds
539,539,Kabwe,ZM,-14.4469,28.4464,46.89,74,0,8.70,clear sky
547,547,Ust-Nera,RU,64.5667,143.2000,52.23,89,100,4.63,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Beloha,MG,59.00,clear sky,-25.1667,45.0500,
1,Severo-Kurilsk,RU,59.34,broken clouds,50.6789,156.1250,
2,Esperance,AU,58.84,clear sky,-33.8667,121.9000,
4,Castro,BR,54.99,clear sky,-24.7911,-50.0119,
5,Sorland,NO,54.14,overcast clouds,67.6670,12.6934,
7,Husavik,IS,51.98,clear sky,66.0449,-17.3389,
9,Hermanus,ZA,50.81,overcast clouds,-34.4187,19.2345,
11,Punta Arenas,CL,33.91,clear sky,-53.1500,-70.9167,
16,Piopio,NZ,57.96,broken clouds,-38.4667,175.0167,
17,Lapua,FI,55.35,overcast clouds,62.9693,23.0088,


In [9]:

# 6a. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6b. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
   # 6c. Set parameters to search for hotels with 5000 meters.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Beloha,MG,59.00,clear sky,-25.1667,45.0500,
1,Severo-Kurilsk,RU,59.34,broken clouds,50.6789,156.1250,Kak Doma
2,Esperance,AU,58.84,clear sky,-33.8667,121.9000,Hospitality Esperance
4,Castro,BR,54.99,clear sky,-24.7911,-50.0119,CHACARA BAILLY
5,Sorland,NO,54.14,overcast clouds,67.6670,12.6934,Sjybrygga fiskecamp
7,Husavik,IS,51.98,clear sky,66.0449,-17.3389,Bjarnabúð
9,Hermanus,ZA,50.81,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
11,Punta Arenas,CL,33.91,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
16,Piopio,NZ,57.96,broken clouds,-38.4667,175.0167,Owls Nest Motel
17,Lapua,FI,55.35,overcast clouds,62.9693,23.0088,Hotelli Lapuahovi


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Severo-Kurilsk,RU,59.34,broken clouds,50.6789,156.1250,Kak Doma
2,Esperance,AU,58.84,clear sky,-33.8667,121.9000,Hospitality Esperance
4,Castro,BR,54.99,clear sky,-24.7911,-50.0119,CHACARA BAILLY
5,Sorland,NO,54.14,overcast clouds,67.6670,12.6934,Sjybrygga fiskecamp
7,Husavik,IS,51.98,clear sky,66.0449,-17.3389,Bjarnabúð


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))